In [1]:
import sys
sys.path.append("../")

In [2]:
import numpy as np
import pandas as pd
import torch
import warnings

from Data.data_sim import SimStudyNonLinearNonPH
from Data.data_sim import SimStudyNonLinearNonPHSquared
from Data.data_sim import SimStudyNonLinearNonPHCubed
from Data.data_sim import SimStudyNonLinearNonPHAll

from pycox import datasets
# from pycox.simulations import SimStudyNonLinearNonPH
from pycox.evaluation import EvalSurv

from model.dataset import Dataset, sample_by_quantiles
from model.fedcox import Federation
from model.net import MLP, MLPPH, CoxPH
from model.discretiser import Discretiser
from model.interpolate import surv_const_pdf_df

from sklearn.model_selection import KFold
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from torch.utils.data import DataLoader

In [3]:
rng = np.random.default_rng(12)
_ = torch.manual_seed(12)

In [4]:
# data = datasets.metabric.read_df()
# data = datasets.support.read_df()
# data = datasets.gbsg.read_df()
data = datasets.flchain.read_df()
# data = datasets.rr_nl_nhp.read_df()
# data.head()

In [5]:
# n = 4000
# sims = [SimStudyNonLinearNonPH(), SimStudyNonLinearNonPHSquared(), SimStudyNonLinearNonPHCubed(), SimStudyNonLinearNonPHAll()]
# sim = sims[3]
# data = sim.simulate(n)
# data = sim.dict2df(data, True)

In [6]:
# data = data.drop(columns=['duration_true','event_true','censoring_true']) # for simulation
data = data.rename(columns={'death' : 'event', 'futime' : 'duration'}) # for flchain

data = data.astype({'event' : int})
data

,age,sex,sample.yr,kappa,lambda,flc.grp,creatinine,mgus,duration,event
0,97.0,0.0,1997,5.700,4.860,10,1.7,0.0,85.0,1
1,92.0,0.0,2000,0.870,0.683,1,0.9,0.0,1281.0,1
2,94.0,0.0,1997,4.360,3.850,10,1.4,0.0,69.0,1
3,92.0,0.0,1996,2.420,2.220,9,1.0,0.0,115.0,1
4,93.0,0.0,1996,1.320,1.690,6,1.1,0.0,1039.0,1
...,...,...,...,...,...,...,...,...,...,...
6519,53.0,0.0,1997,0.705,1.250,2,0.8,0.0,4547.0,0
6520,52.0,0.0,1996,0.786,1.030,2,0.7,0.0,4788.0,0
6521,52.0,0.0,1995,1.210,1.610,6,1.0,0.0,4997.0,0
6522,52.0,0.0,1999,0.858,0.581,1,0.8,0.0,3652.0,0


In [7]:
# standardisation of features
# simulation
# cols_minmax = ['x0', 'x1', 'x2']
# all_cols = cols_minmax 

# metabric
# cols_minmax = ['x0', 'x1', 'x2', 'x3','x8']
# cols_leave = ['x4','x5','x6','x7']
# all_cols = cols_minmax + cols_leave
# leave = [(col, None) for col in cols_leave]

# support
# cols_minmax = ['x0','x2','x3','x6','x7', 'x8', 'x9','x10','x11','x12','x13']
# cols_leave = ['x1','x4','x5']
# all_cols = cols_minmax + cols_leave
# leave = [(col, None) for col in cols_leave]

# gbsg
# cols_minmax = ['x3', 'x4','x5', 'x6']
# cols_leave = ['x0','x1','x2']
# all_cols = cols_minmax + cols_leave
# leave = [(col, None) for col in cols_leave]

# flchain
cols_minmax = ['age','sample.yr','kappa','lambda','flc.grp','creatinine']
cols_leave = ['mgus','sex']
all_cols = cols_minmax + cols_leave
leave = [(col, None) for col in cols_leave]

minmax = [([col], MinMaxScaler()) for col in cols_minmax] # ok for all

# x_mapper = DataFrameMapper(minmax) # simulation
x_mapper = DataFrameMapper(minmax + leave) # all other datasets

# discretisation
num_durations = 10
discretiser = Discretiser(num_durations, scheme='km')

In [8]:
def train_val_split(df, t_index, v_index, x_mapper, fit_transform=True):
    df_t = df.loc[t_index]
    df_v = df.loc[v_index]

    if fit_transform:
        x_t = x_mapper.fit_transform(df_t).astype('float32')
    else:
        x_t = x_mapper.transform(df_t).astype('float32')
    x_v = x_mapper.transform(df_v).astype('float32')

    y_t = (df_t.duration.values, df_t.event.values)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        if fit_transform:
            y_t = discretiser.fit_transform(*y_t)
        else:
            y_t = discretiser.transform(*y_t)

    y_v = (df_v.duration.values, df_v.event.values)

    return x_t, y_t, x_v, y_v, df_t.duration.values #ADDED df_t

In [9]:
case1 = {'num_centers' : 1, 
            'local_epochs' : [1],
            'stratify_labels' : False,
            'case_id' : 'central'}

case2 = {'num_centers' : 4, 
            'local_epochs' : [1,5,20,100],
            'stratify_labels' : False,
            'case_id' : 'iid'}

case3 = {'num_centers' : 4, 
            'local_epochs' : [1,5,20,100],
            'stratify_labels' : True,
            'case_id' : 'noniid'}

### just for val losses
# if True:
#     case2 = {'num_centers' : 4, 
#                 'local_epochs' : [1],
#                 'stratify_labels' : False,
#                 'case_id' : 'iid'}

#     case3 = {'num_centers' : 4, 
#                 'local_epochs' : [1],
#                 'stratify_labels' : True,
#                 'case_id' : 'noniid'}
##

cases = [case1, case2, case3]
# cases = [case3]
 
model_type = 'NNnph'
loss_folder = f'../results-flchain/losses'
log_folder = f'../results-flchain/{model_type}'
test_by_center = True


reset_in = 6 

for case in cases:

    # if equal to 1 only once and only on the first fold of case 1, if equal to ev-folds times para-folds then every time in case 1
    tune_tries = 5
    para_round = 0

    best_lr = 0.01
    best_dropout = 0
    tuning = True

    reset_in = reset_in - 1
    if reset_in == 0:
        rng = np.random.default_rng(12)
        _ = torch.manual_seed(12)  
        reset_in = 6      

    case_id = case['case_id']
    
    # federation parameters - excl lr
    num_centers = case['num_centers']
    optimizer = 'adam'
    batch_size = 256
    local_epochs = 1 # overridden below
    base_epochs = 100
    print_every = 100
    # no stratification if None and False
    stratify_col = None
    stratify_labels = case['stratify_labels']

    # this is set automatically
    stratify_on = None

    if stratify_col != None:
        stratify_on = all_cols.index(stratify_col)
        print(f'Stratify on index: {stratify_on}')
    if stratify_labels:
        stratify_on = 0
        print(f'Stratify on label index: {stratify_on}')
        
    # case level
    for local_epochs in case['local_epochs']:
        
        epochs = max(1, base_epochs // local_epochs)

        log = f'{log_folder}/training_log_M{model_type}C{case_id}S{stratify_on}C{num_centers}L{local_epochs}.txt'
        with open(log, 'w') as f:
            print(f'-- Centers: {num_centers}, Local rounds: {local_epochs}, Global rounds: {epochs} --', file=f)

        case_local_val_losses = []
        case_global_val_losses = []
        case_local_train_losses = []
        case_global_train_losses = []

        # CV setup
        n_splits = 5
        random_state = rng.integers(0,1000)
        scores = []
        briers = []
        parameters = []

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        cv_round = 0
        
        # CV for average performance
        for train_index, test_index in kf.split(data):
            with open(log, 'a') as f:
                print(f'-- Eval CV fold: {cv_round} --', file=f)
            cv_round += 1
            x_train, y_train, x_test, y_test, df_train = train_val_split(data, train_index, test_index, x_mapper, fit_transform=True)
            test_loader = DataLoader(Dataset(x_test, y_test), batch_size=256, shuffle=False)

            # MLP parameters - excl dropout
            dim_in = x_train.shape[1]
            num_nodes = [32, 32]
            dim_out = len(discretiser.cuts)
            batch_norm = True

            # tuning
            if tuning:
                # grid for parameter 1 to be tuned
                learning_rates = [0.1, 0.01, 0.001, 0.0001]

                # grid for parameter 2 to be tuned
                # >>> for CoxPH just set to 0 - doesn't make a difference
                # dropouts = [0.1, 0.5, 0.75] 
                dropouts = [0]
                
                # [[scores for each lr x dropout from fold 1], [..from fold2], etc.]
                tuning_scores = []    

                para_splits = 5
                para_kf = KFold(n_splits=para_splits)
                for t_index, v_index in kf.split(x_train):
                    
                    x_t, y_t, x_v, y_v, df_t = train_val_split(data.loc[train_index].reset_index(), t_index, v_index, x_mapper, fit_transform=False)

                    val_loader = DataLoader(Dataset(x_v, y_v), batch_size=256, shuffle=False)

                    # each entry corresponds to the score for a particular lr x dropout pair
                    fold_scores = []
                    for lr in learning_rates:
                        for dropout in dropouts:
                            
                            para_epochs = max(1,epochs // 5)

                            if model_type == 'NNnph':   
                                net = MLP(dim_in=dim_in, num_nodes=num_nodes, dim_out=dim_out, batch_norm=batch_norm, dropout=dropout)
                            if model_type == 'CoxPH':
                                net = CoxPH(dim_in=dim_in, dim_out=dim_out, batch_norm=batch_norm)
                            if model_type == 'NNph':
                                net = MLPPH(dim_in=dim_in, num_nodes=num_nodes, dim_out=dim_out, batch_norm=batch_norm, dropout=dropout)
                            else:
                                ValueError

                            fed = Federation(features=x_t, labels=y_t, net=net, num_centers=num_centers, optimizer=optimizer, lr=lr, stratify_on=stratify_on, stratify_labels=stratify_labels, batch_size=batch_size, local_epochs=local_epochs,df_t=df_t)
                            ran_for = fed.fit(epochs=para_epochs, patience=999, print_every=print_every, take_best=True, verbose=False)    
                            # ran_for = fed.fit(epochs=para_epochs, patience=999, print_every=print_every, take_best=False, verbose=False)    

                            surv = fed.predict_surv(val_loader)[0]
                            surv = surv_const_pdf_df(surv, discretiser.cuts) # interpolation

                            ev = EvalSurv(surv, *y_v, censor_surv='km')
                            score = ev.concordance_td('antolini')
                            fold_scores.append(score)
                            with open(log, 'a') as f:
                                print(f'Tuning CV fold {para_round} with {ran_for} rounds: conc = {score}, lr = {lr}, dropout = {dropout}', file=f)
                    tuning_scores.append(fold_scores)
                    
                    para_round += 1
                    if para_round >= tune_tries:
                        tuning = False
                        break # out of para loop

                tuning_scores = np.array(tuning_scores)
                avg_scores = np.mean(tuning_scores, axis=0)
                best_combo_idx = np.argmax(avg_scores)
                best_lr_idx = best_combo_idx // len(dropouts)
                best_dropout_idx = best_combo_idx % len(dropouts)
                best_lr = learning_rates[best_lr_idx]
                best_dropout = dropouts[best_dropout_idx]

            if model_type == 'NNnph':   
                net = MLP(dim_in=dim_in, num_nodes=num_nodes, dim_out=dim_out, batch_norm=batch_norm, dropout=best_dropout)    
            if model_type == 'CoxPH':            
                net = CoxPH(dim_in=dim_in, dim_out=dim_out, batch_norm=batch_norm)
            if model_type == 'NNph':
                net = MLPPH(dim_in=dim_in, num_nodes=num_nodes, dim_out=dim_out, batch_norm=batch_norm, dropout=best_dropout)
            else:
                ValueError

            fed = Federation(features=x_train, labels=y_train, net=net, num_centers=num_centers, optimizer=optimizer, lr=best_lr, stratify_on=stratify_on, stratify_labels=stratify_labels, batch_size=batch_size, local_epochs=local_epochs,df_t=df_train)
            ran_for = fed.fit(epochs=epochs, patience=999, print_every=print_every, take_best=True)    
            # ran_for = fed.fit(epochs=epochs, patience=999, print_every=print_every, take_best=False)    
            
            surv = fed.predict_surv(test_loader)[0]
            surv = surv_const_pdf_df(surv, discretiser.cuts) # interpolation
            
            time_grid = np.linspace(y_test[0].min(), y_test[0].max(), 100)
            
            if test_by_center:
                dict_center_idxs_test = sample_by_quantiles(y_test,0,4,y_test[0])
                for center in dict_center_idxs_test:
                    idxs_test = dict_center_idxs_test[center]
                    ev = EvalSurv(surv.iloc[:, idxs_test], y_test[0][idxs_test], y_test[1][idxs_test], censor_surv='km')
                    score = ev.concordance_td('antolini')
                    brier = ev.integrated_brier_score(time_grid) 
                    with open(log, 'a') as f:
                        print(f'>> Center {center}: conc = {score}, brier = {brier}, LR = {best_lr}, dropout = {best_dropout}', file=f)
            ev = EvalSurv(surv, *y_test, censor_surv='km')
            score = ev.concordance_td('antolini')
            scores.append(score)

            brier = ev.integrated_brier_score(time_grid) 
            briers.append(brier)
            with open(log, 'a') as f:
                print(f'>> After {ran_for} rounds, model from round {fed.model_from_round}: conc = {score}, brier = {brier}, LR = {best_lr}, dropout = {best_dropout}', file=f)

            parameters.append({'lr' : best_lr, 'dropout' : best_dropout})
            case_local_val_losses.append(fed.local_val_losses)
            case_global_val_losses.append(fed.global_val_losses)
            case_local_train_losses.append(fed.local_train_losses)
            case_global_train_losses.append(fed.global_train_losses)


        losses = np.array(case_local_val_losses)
        lossfile = f'{loss_folder}/local_val_loss_M{model_type}C{case_id}L{local_epochs}.npy'
        np.save(lossfile, losses)

        losses = np.array(case_global_val_losses)
        lossfile = f'{loss_folder}/global_val_loss_M{model_type}C{case_id}L{local_epochs}.npy'
        np.save(lossfile, losses)

        losses = np.array(case_local_train_losses)
        lossfile = f'{loss_folder}/local_train_loss_M{model_type}C{case_id}L{local_epochs}.npy'
        np.save(lossfile, losses)

        losses = np.array(case_global_train_losses)
        lossfile = f'{loss_folder}/global_train_loss_M{model_type}C{case_id}L{local_epochs}.npy'
        np.save(lossfile, losses)

        with open(log, 'a') as f:
            print(f'Avg concordance: {sum(scores) / len(scores)}, Integrated Brier: {sum(briers) / len(briers)}', file=f)


4175
Epochs exhausted, model from round 13
4175
Epochs exhausted, model from round 5
4175
Epochs exhausted, model from round 20
4175
Epochs exhausted, model from round 20
4175
Epochs exhausted, model from round 20
4175
Epochs exhausted, model from round 5
4175
Epochs exhausted, model from round 20
4175
Epochs exhausted, model from round 20
4175
Epochs exhausted, model from round 14
4175
Epochs exhausted, model from round 18
4175
Epochs exhausted, model from round 20
4175
Epochs exhausted, model from round 20
4175
Epochs exhausted, model from round 10
4175
Epochs exhausted, model from round 8
4175
Epochs exhausted, model from round 20
4175
Epochs exhausted, model from round 20
4176
Epochs exhausted, model from round 17
4176
Epochs exhausted, model from round 5
4176
Epochs exhausted, model from round 20
4176
Epochs exhausted, model from round 20
5219
 \Latest training stats after 100 global rounds:
Training loss : 1.1080637040891146
Validation loss : 0.3447552830690429
Epochs exhausted, 

TypeError: object of type 'NoneType' has no len()